In [1]:
import cv2
from ultralytics import YOLOWorld
import supervision as sv
import os

In [2]:
# Путь к изображениям
PATH_TO_IMAGES = "sample_substraction"
# Размер батча
BATCH_SIZE = 5
# Размер изображения
image_size = (1280, 736)
# Порог для фильтрации детекций (30% от размера изображения)
threshold_of_detection_size = 0.3
# Список картинок
list_of_images = os.listdir(PATH_TO_IMAGES)
list_of_images_with_relative_path = [os.path.join(PATH_TO_IMAGES, image) for image in list_of_images]

In [3]:
# Инициализируем модель
model = YOLOWorld("yolov8x-worldv2.pt")

# Задаем промпт для модели
classes = ["license_plate", ""]
model.set_classes(classes)

# Создаем датасет
dataset = sv.DetectionDataset(classes=classes, 
                              images=list_of_images_with_relative_path, 
                              annotations={list_of_images_with_relative_path[i]: None 
                                           for i in range(len(list_of_images_with_relative_path))})

# Итерация по списку картинок: обрабатываем картинки батчами
for i in range(0, len(list_of_images_with_relative_path), BATCH_SIZE):
    batch = list_of_images_with_relative_path[i:BATCH_SIZE+i+1]
    lst_of_cv2_instances = [cv2.imread(image) for image in batch]

    # Запускаем детекцию (подаем батч картинок) с порогом уверенности
    results = model.predict(lst_of_cv2_instances, conf=0.1, imgsz=image_size)

    # Итерация по каждой детекции в батче
    for index in range(len(results)):
        # Получение детекций
        detections = sv.Detections.from_ultralytics(results[index])

        # Считаем кол-во пикселей на изображении
        image_area = image_size[0] * image_size[1]

        # фильтрация детекций по ширине каждого найденного bounding-box-а
        w = detections.xyxy[:, 2] - detections.xyxy[:, 0]
        h = detections.xyxy[:, 3] - detections.xyxy[:, 1]

        # Подрезаем детекции, которые занимают более чем 30% изображения
        detections = detections[(w * h) / image_area < threshold_of_detection_size]

        #  Non-Maximum Suppression (NMS) постобработка для удаления дублирующихся прямоугольников
        filtered_detections = detections.with_nms(threshold = 1e-9)

        # Инициализация экземпляров классов для боксов и подписей
        annotator = sv.BoxAnnotator()
        label_annotator = sv.LabelAnnotator()

        # вывод доп информации (уверенность)
        labels = [
            f"{class_name} {confidence:.2f}"
            for class_name, confidence
            in zip(filtered_detections['class_name'], filtered_detections.confidence)
        ]

        # Непосредственно, рисуем боксы и подписи
        annotated_image = annotator.annotate(scene = lst_of_cv2_instances[index], 
                                            detections = filtered_detections)
        annotated_image = label_annotator.annotate(scene = annotated_image, 
                                                detections = filtered_detections, 
                                                labels=labels)
        # Сохраняем размеченную картинку
        cv2.imwrite(f"annotated_{batch[index]}", annotated_image)

        # Вносим в датасет инфу по детекциям для конкретной картинки
        dataset.annotations[batch[index]] = filtered_detections



0: 1280x736 2 license_plates, 85.6ms
1: 1280x736 2 license_plates, 85.6ms
2: 1280x736 4 license_plates, 85.6ms
3: 1280x736 3 license_plates, 85.6ms
4: 1280x736 4 license_plates, 85.6ms
5: 1280x736 5 license_plates, 85.6ms
Speed: 3.5ms preprocess, 85.6ms inference, 13.7ms postprocess per image at shape (1, 3, 1280, 736)

0: 1280x736 5 license_plates, 81.3ms
1: 1280x736 3 license_plates, 81.3ms
2: 1280x736 1 license_plate, 81.3ms
3: 1280x736 2 license_plates, 81.3ms
4: 1280x736 4 license_plates, 81.3ms
5: 1280x736 2 license_plates, 81.3ms
Speed: 3.7ms preprocess, 81.3ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 736)

0: 1280x736 2 license_plates, 81.5ms
1: 1280x736 2 license_plates, 81.5ms
2: 1280x736 2 license_plates, 81.5ms
3: 1280x736 3 license_plates, 81.5ms
4: 1280x736 2 license_plates, 81.5ms
5: 1280x736 2 license_plates, 81.5ms
Speed: 3.5ms preprocess, 81.5ms inference, 1.1ms postprocess per image at shape (1, 3, 1280, 736)

0: 1280x736 2 license_plates, 81.4ms

In [7]:
# Экспортируем в YOLO-формат (data_yaml генерируется невалидный для CVAT, нужно создавать самому)
dataset.as_yolo(
    images_directory_path="subbatch_license_plate_dataset/images/train",
    annotations_directory_path="subbatch_license_plate_dataset/labels/train",
)